# Query data for Monterrey Mexico from OpenAgua published in HydroShare

#### By Adel M. Abdallah, Utah State University, January 2018

Execute the following cells by pressing `Shift-Enter`, or by pressing the play button <img style='display:inline;padding-bottom:15px' src='play-button.png'> on the toolbar above.



<a name="Import"></a>
# 1. Import python libraries 

In [ ]:
# 1. Import python libraries 
### set the notebook mode to embed the figures within the cell

import sqlite3
import numpy as np
import pandas as pd
import getpass
from hs_restclient import HydroShare, HydroShareAuthBasic
import os

import plotly
plotly.__version__
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
offline.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)         # initiate notebook for offline plot

import os
import csv
from collections import OrderedDict
import sqlite3
import pandas as pd
import numpy as np
from IPython.display import display, Image, SVG, Math, YouTubeVideo
import urllib
import calendar

print 'The needed Python libraries have been imported'

# 2. Connect to the WaMDaM SQLite on HydroSahre
### Provide the HydroShare ID for your resource
Example  
https://www.hydroshare.org/resource/e29c9283305045338be24a495c781ec9/


In [ ]:
username = 'amabdallah'
password = 'MyHydroShareWorld'

auth = HydroShareAuthBasic(username=username, password=password)

hs = HydroShare(auth=auth)

print 'Connected to HydroShare'

# Then we can run queries against it within this notebook :)  
resource_url='https://www.hydroshare.org/resource/e29c9283305045338be24a495c781ec9/' 


resource_id= resource_url.split("https://www.hydroshare.org/resource/",1)[1] 
resource_id=resource_id.replace('/','')

print resource_id

resource_md = hs.getSystemMetadata(resource_id)
# print resource_md
print 'Resource title'
print(resource_md['resource_title'])
print '----------------------------'

resources=hs.resource(resource_id).files.all()

file = ""

for f in hs.resource(resource_id).files.all():

    file += f.decode('utf8')

import json

file_json = json.loads(file)

for f in file_json["results"]:

    FileURL= f["url"]
    
    SQLiteFileName=FileURL.split("contents/",1)[1] 

cwd = os.getcwd()
print cwd
fpath = hs.getResourceFile(resource_id, SQLiteFileName, destination=cwd)
conn = sqlite3.connect(SQLiteFileName,timeout=10)

print 'Connected to the SQLite file= '+ SQLiteFileName
print 'done'

<a name="QuerySupplyDataLoadWEAP"></a>
# Query delivery target and obseved flow at DR Bajo Rio San Juan demand site
 
### The data comes from OpenAgua

In [ ]:
# Use Case 3.1Identify_aggregate_TimeSeriesValues.csv
# plot aggregated to monthly and converted to acre-feet time series data of multiple sources



# 2.2Identify_aggregate_TimeSeriesValues.csv
Query_UseCase3_1_URL="""
https://raw.githubusercontent.com/WamdamProject/WaMDaM_JupyterNotebooks/master/3_VisualizeShare/Monterrey_Mexico/Delivery_demand_TS_DR_Bajo_Rio_San_Juan.sql
"""

# Read the query text inside the URL
Query_UseCase3_1_text = urllib.urlopen(Query_UseCase3_1_URL).read()


# return query result in a pandas data frame
result_df_UseCase3_1= pd.read_sql_query(Query_UseCase3_1_text, conn)

# uncomment the below line to see the list of attributes
display (result_df_UseCase3_1)

# print result_df_UseCase3_1.keys()
print "Query is done"

## Plot and compare demand and observed delievery for the baseline and calibration scenariosin Monterrey model

In [ ]:

df_TimeSeries=result_df_UseCase3_1
# identify the data for four time series only based on the DatasetAcronym column header 
column_name = ["ScenarioName","AttributeName"]
subsets = df_TimeSeries.groupby(column_name)
data = []
subsets_settings = {
    ('Baseline','Observed Delivery'): {
        'dash': 'solid',
        'legend_index': 0,
        'legend_name': 'Baseline: Observed Delivery',
        'width':2,
        'color':'rgb(0, 0, 0)'
        },
    ('Baseline','Demand'): {
        'dash': 'solid',
        'legend_index': 2,
        'legend_name': 'Baseline: Demand',
        'width':2,
        'color':'rgb(15, 107, 153)'
        },
    ('Calibration','Observed Delivery'): {
        'dash': 'dash',
        'legend_index': 1,
        'legend_name': 'Calibration: Observed Delivery',
        'width':2,
        'color':'rgb(0, 0, 0)'
        },    
    ('Calibration','Demand'): {
        'dash': 'dot',     # this is properity of the line (curve)
        'legend_index': 3,   # to order the legend
        'legend_name': 'Calibration: Demand',
        'width':2,
        'color':'rgb(107, 153, 15)'
        }
    }

# This dict is used to map legend_name to original subset name
subsets_names = {y['legend_name']: x for x,y in subsets_settings.iteritems()}

for subset in subsets.groups.keys():
    dt = subsets.get_group(name=subset)
    print subset
#     if subset == ('Baseline','Observed Delivery'):

    s = go.Scatter(
                    x=dt['DateTimeStamp'],
                    y=dt['DataValue'],
                    name = subsets_settings[subset]['legend_name'],
                    line = dict(
                        color =subsets_settings[subset]['color'],
                        width =subsets_settings[subset]['width'], 
                        dash=subsets_settings[subset]['dash']
                               ),
                        opacity = 1                                
                  )
    data.append(s)
    
# Legend is ordered based on data, so we are sorting the data based 
# on desired legend order indicarted by the index value entered above
data.sort(key=lambda x: subsets_settings[subsets_names[x['name']]]['legend_index'])
    
    
layout = dict(
#title = "Use Case 3.3",
yaxis = dict(
    title = "Flow <br> cubic meter/sec.",            automargin=True,

#     tickformat= ',',

        showline=True,
        dtick='0.25',
        ticks='outside',
        ticklen=10,
         range = ['0', '1.25']


            ),

xaxis = dict(
#         title = "Updated input parameters in the <br>Bear_River_WEAP_Model_2017",
#         showline=True,
    ticks='outside',            automargin=True,
#          range = ['1990','2020']

#          tickfont=dict(size=22),
        ticklen=20
                ),
legend=dict(
        x=0.05,y=0.9,
      bordercolor='#00000f',
        borderwidth=2,

           ),
    width=1100,
    height=700,

 margin=dict(l=150,pad=4),
font=dict(size=25,family='arial',color='#00000f'),
showlegend=True,
    
)


# create a figure object
fig = dict(data=data, layout=layout)
#py.iplot(fig, filename = "2.3Identify_SeasonalValues") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
offline.iplot(fig,filename = 'Monterrey' )       

print "Figure x is replicated!!"

<a name="Close"></a>
# 7. Close the SQLite connection

In [ ]:
# 9. Close the SQLite and WEAP API connections
conn.close()

print 'connection disconnected'



# The End :) Congratulations!